In [1]:
from context import omphalos

import numpy as np
import matplotlib as pyplot
# Import Omphalos modules.
from omphalos import generate_inputs as gi
from omphalos import file_methods as fm
from omphalos import my_metrics as mm
from omphalos import omphalos_plotter as op
from omphalos import attributes as attr
from omphalos import labels as lbls
import sys
import subprocess
import pickle

import warnings
import numpy as np
import gpflow
import tensorflow as tf

from matplotlib import pyplot as plt

from gpflow.ci_utils import ci_niter, ci_range
from gpflow.models import VGP, GPR, SGPR, SVGP
from gpflow.optimizers import NaturalGradient
from gpflow.optimizers.natgrad import XiSqrtMeanVar
from gpflow import set_trainable

%matplotlib inline
%precision 4

np.random.seed(0)
tf.random.set_seed(0)

N, D = 100, 2
batch_size = 50

# inducing points
M = 10

inducing_variable = tf.random.uniform((M, D))
adam_learning_rate = 0.01
iterations = ci_niter(5)
autotune = tf.data.experimental.AUTOTUNE

In [2]:
train_set = fm.unpickle('data/rifle_pyrite.pickle')

attributes_df = attr.boundary_condition(train_set, boundary='x_begin')
labels_df = lbls.secondary_precip(train_set)

/usr/local/lib/python3.8/site-packages/pandas/core/frame.py:6390: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = expressions.where(mask, this, that)


In [3]:
x = attributes_df.loc[:, ['NH4+', 'SO4--','Ca++', 'Acetate', 'CO2(aq)']]
y = labels_df.sum(level=0)['FeS(am)'] + labels_df.sum(level=0)['FeS34(am)']

x = x.to_numpy() 
y = y.to_numpy().reshape(-1,1)
y = y * 1e4
x = np.log(x)

data = (x,y)

In [4]:
%%time

vgp = VGP(data, kernel=gpflow.kernels.Matern52(lengthscales=[1,1,1,1,1], active_dims=[0,1,2,3,4]), likelihood=gpflow.likelihoods.Gaussian())
vgp.elbo().numpy()

natgrad_opt = NaturalGradient(gamma=1.0)
variational_params = [(vgp.q_mu, vgp.q_sqrt)]
natgrad_opt.minimize(vgp.training_loss, var_list=variational_params)
vgp.elbo().numpy()

from gpflow.utilities import print_summary
print_summary(vgp)

╒═════════════════════════╤═══════════╤══════════════════╤═════════╤═════════════╤═══════════════════╤═════════╤══════════════════════════════════════════╕
│ name                    │ class     │ transform        │ prior   │ trainable   │ shape             │ dtype   │ value                                    │
╞═════════════════════════╪═══════════╪══════════════════╪═════════╪═════════════╪═══════════════════╪═════════╪══════════════════════════════════════════╡
│ VGP.kernel.variance     │ Parameter │ Softplus         │         │ True        │ ()                │ float64 │ 1.0                                      │
├─────────────────────────┼───────────┼──────────────────┼─────────┼─────────────┼───────────────────┼─────────┼──────────────────────────────────────────┤
│ VGP.kernel.lengthscales │ Parameter │ Softplus         │         │ True        │ (5,)              │ float64 │ [1., 1., 1....                           │
├─────────────────────────┼───────────┼──────────────────┼──────

In [5]:
%%time

plot_var=1
# We define a function that plots the model's prediction (in the form of samples) together with the data.
# Importantly, this function has no other argument than `fig: matplotlib.figure.Figure` and `ax: matplotlib.figure.Axes`.
X=x
Y=y

# plot the function posterior
samp = 50
xx = np.ones((samp, 5))
xx[:,0] = 1.5
xx[:,1] = 8.8
xx[:,2] = 20
xx[:,3] = 9.7
xx[:,4] = 0.0325
xx[:,plot_var] =  np.linspace(1e-7, 30, samp)

xx = np.log(xx)

Xnew=xx

mean, var = vgp.predict_f(xx)

CPU times: user 28.8 s, sys: 20 s, total: 48.8 s
Wall time: 29.1 s


In [ ]:
sweep = fm.unpickle('data/rifle_sweeps/rifle_so4_sweep.pickle')

sweep_attr = attr.boundary_condition(sweep, boundary='x_begin')
sweep_lbls = lbls.secondary_precip(sweep)

sweep_x = sweep_attr.loc[:, 'SO4--']
sweep_y = sweep_lbls.sum(level=0)['FeS(am)'] + sweep_lbls.sum(level=0)['FeS34(am)']
sweep_y = sweep_y

In [ ]:
sweep[0].condition_blocks['amendment'].primary_species

In [ ]:
def plot_prediction(fig, ax):
    #Ypred = vgp.predict_f_samples(Xnew, full_cov=True, num_samples=20)
    #ax.plot(np.exp(Xnew[:,1]), np.squeeze(Ypred).T, "C1", alpha=0.2)
    ax.scatter(sweep_x, sweep_y, c="k", marker='+', s=100,label='RTM results')
    ax.plot(np.exp(Xnew[:, plot_var]), mean * 1e-4, label='GPR fit')
    ax.fill_between(np.exp(Xnew[:, plot_var]), (mean[:,0] - 2 * np.sqrt(var[:,0])) * 1e-4, (mean[:,0] + 2 * np.sqrt(var[:,0])) * 1e-4, color="C0", alpha=0.2, label='95% confidence interval')

axis_labels = ['[NH$_4^+$] (mM)', '[SO$_4^{2-}$] (mM)','[Ca$^2+$] (mM)', '[Acetate] (mM)', '[CO$_{2(aq)}$] (bar)']

# Let's check if the function does the desired plotting
fig = plt.figure(figsize=(16,9))
ax = fig.subplots()
plot_prediction(fig, ax)
ax.set_xlabel(axis_labels[plot_var], fontsize=20)
ax.set_ylabel('Net pyrite precipitation (vol. frac.)', fontsize=20)
ax.legend(fontsize=20)
ax.tick_params('both', labelsize=18)
ax.set_xlim(0, 30)
ax.set_ylim(0, 0.0040)


path = '/Users/angus/PhD_pres/fit_plots/rifle{}'.format(plot_var)
fig.savefig(path, dpi=300)

In [27]:
%%time

num = 20
upper = 30
lower = 1e-5
x1 = np.linspace(lower, upper, num)
x2 = np.linspace(lower, upper, num)
x1_mesh, x2_mesh = np.meshgrid(x1, x2)
X = np.dstack([x1_mesh, x2_mesh]).reshape(-1, 2)

control_vals = np.zeros((num**2, 5))

control_vals[:,0] = 1.5 # NH4+
control_vals[:,1] = 8.8 # SO4--
control_vals[:,2] = 20.0 # Ca++  
control_vals[:,3] = 9.7 # Acetate
control_vals[:,4] = 0.0325 # CO2


XX = np.hstack((control_vals[:,0].reshape(-1,1), X[:,0].reshape(-1,1), control_vals[:,2].reshape(-1,1), X[:,1].reshape(-1,1), control_vals[:,4].reshape(-1,1)))

vals, var = vgp.predict_f(np.log(XX))

CPU times: user 37.1 s, sys: 24.3 s, total: 1min 1s
Wall time: 37.8 s


In [24]:
%%time

num = 20
upper = 30
lower = 1e-5
x1 = np.linspace(lower, upper, num)
x2 = np.linspace(lower, 10, num)
x1_mesh, x2_mesh = np.meshgrid(x1, x2)
X = np.dstack([x1_mesh, x2_mesh]).reshape(-1, 2)

control_vals = np.zeros((num**2, 5))

control_vals[:,0] = 1.5 # NH4+
control_vals[:,1] = 8.8 # SO4--
control_vals[:,2] = 20.0 # Ca++  
control_vals[:,3] = 9.7 # Acetate
control_vals[:,4] = 0.0325 # CO2


XX = np.hstack((control_vals[:,0].reshape(-1,1), X[:,0].reshape(-1,1), control_vals[:,2].reshape(-1,1), control_vals[:,3].reshape(-1,1), X[:,1].reshape(-1,1)))

vals, var = vgp.predict_f(np.log(XX))

CPU times: user 35.6 s, sys: 20.5 s, total: 56.2 s
Wall time: 32 s


In [15]:
%%time

#Finding best parameter set!
num = 7
upper = 30
lower = 1e-5
x1 = np.linspace(lower, upper, num)
x2 = np.linspace(lower, upper, num)
x3 = np.linspace(lower, upper, num)
x4 = np.linspace(lower, upper, num)
x5 = np.linspace(lower, 10, num)

X = np.stack(np.meshgrid(x1, x2, x3, x4, x5), -1).reshape(-1, 5)

vals, var = vgp.predict_f(np.log(X))

max_index = np.argmax(vals)

print(max_index)
print(X[max_index])

vals[max_index] * 1e-4

In [30]:
%matplotlib widget
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

plt.style.use('seaborn-talk')


axis_labels =  ['[SO$_4^{2-}$] (mM)','[Ca$^{2+}$] (mM)', '[Acetate] (mM)', '[CO$_{2(aq)}$] (bar)']


fig = plt.figure(figsize=(20,20))
ax = fig.add_subplot(111, projection='3d')
ax.set_xlabel(axis_labels[2], fontsize=20, labelpad=20)
ax.set_ylabel(axis_labels[0], fontsize=20, labelpad=20)
ax.set_zlabel('Net pyrite precipitation (vol. frac.)', fontsize=20, labelpad=30)
ax.tick_params('both', labelsize=18)
ax.ticklabel_format(axis='z', style='sci', scilimits=(0,0))
ax.set_zlim(-0.5e-3, 5e-3)
ax.view_init(elev=28, azim=-111)
def shape(obj):
    print(np.shape(obj))
    

ax.plot_trisurf(XX[:,3], XX[:,1], vals.numpy().reshape(-1) * 1e-4,  cmap=plt.cm.magma)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [31]:
fig.savefig('/Users/angus/PhD_pres/3D_plots/rifle_acetate_so4', dpi=300)